In [1]:
import pandas as pd
import gzip
import os
import pickle

# Leer el archivo
with gzip.open('clinvar.vcf.gz', 'rt') as f:
    lines = [line for line in f if not line.startswith('##')]
    df = pd.DataFrame([line.strip().split('\t') for line in lines])
    df.columns = df.iloc[0].str.replace('^#', '', regex=True)
    df = df[1:]

# Seleccionar columnas relevantes
df = df[['CHROM', 'POS', 'ID', 'REF', 'ALT', 'INFO']]

# Función para parsear el contenido de la columna INFO
def parse_info(info_string):
    info_dict = {}
    for item in info_string.split(';'):
        if '=' in item:
            key, value = item.split('=', 1)
            info_dict[key] = value
    return info_dict

# Aplicar la función de parseo
df = pd.concat([df.drop('INFO', axis=1), df['INFO'].apply(parse_info).apply(pd.Series)], axis=1)

# Procesar la columna MC si está presente
if 'MC' in df.columns:
    def process_first_mc(mc_string):
        if pd.notna(mc_string):
            first_entry = mc_string.split(';')[0]  # assuming MC entries are separated by ';'
            if '|' in first_entry:
                identifier, consequence = first_entry.split('|', 1)
                return pd.Series([identifier, consequence])
            else:
                return pd.Series([None, None])
        else:
            return pd.Series([None, None])
    df[['MC_SYMBOL', 'MC_CONSEQUENCE']] = df['MC'].apply(process_first_mc)

# Procesar GENEINFO si está presente
if 'GENEINFO' in df.columns:
    df['GENEINFO'] = df['GENEINFO'].apply(lambda x: x.split('|')[0] if pd.notna(x) and '|' in x else x)
    df[['GENE_SYMBOL', 'GENE_ID']] = df['GENEINFO'].str.split(':', expand=True)

# Tomar una muestra del 0.01 del dataframe
df_sample = df.sample(frac=0.01 , random_state=1)


In [2]:
# Guardar el DataFrame con pickle
pickle_filename = 'final_dataframe_simplified.pkl'
with open(pickle_filename, 'wb') as file:
    pickle.dump(df_sample, file)

print(f"DataFrame guardado con éxito en {pickle_filename}.")

In [3]:
with open('final_dataframe.pkl', 'rb') as file:
    loaded_df = pickle.load(file)

In [4]:
chro_uri_mapping = {
    1: 'NC_000001.11',
    2: 'NC_000002.12',
    3: 'NC_000003.12',
    4: 'NC_000004.12',
    5: 'NC_000005.10',
    6: 'NC_000006.12',
    7: 'NC_000007.14',
    8: 'NC_000008.11',
    9: 'NC_000009.12',
    10: 'NC_000010.11',
    11: 'NC_000011.10',
    12: 'NC_000012.12',
    13: 'NC_000013.11',
    14: 'NC_000014.9',
    15: 'NC_000015.10',
    16: 'NC_000016.10',
    17: 'NC_000017.11',
    18: 'NC_000018.10',
    19: 'NC_000019.10',
    20: 'NC_000020.11',
    21: 'NC_000021.9',
    22: 'NC_000022.11',
    'X': 'NC_000023.11',
    'Y': 'NC_000024.10',
    'NT_113889.1': 'NT_113889.1',
    'NT_187633.1': 'NT_187633.1',
    'NT_187661.1': 'NT_187661.1',
    'NT_187693.1': 'NT_187693.1',
    'NW_009646201.1': 'NW_009646201.1'
}

In [5]:
descrip_alt_label_mapping = {
    1: 'NC_000001.11:g.',
    2: 'NC_000002.12:g.',
    3: 'NC_000003.12:g.',
    4: 'NC_000004.12:g.',
    5: 'NC_000005.10:g.',
    6: 'NC_000006.12:g.',
    7: 'NC_000007.14:g.',
    8: 'NC_000008.11:g.',
    9: 'NC_000009.12:g.',
    10: 'NC_000010.11:g.',
    11: 'NC_000011.10:g.',
    12: 'NC_000012.12:g.',
    13: 'NC_000013.11:g.',
    14: 'NC_000014.9:g.',
    15: 'NC_000015.10:g.',
    16: 'NC_000016.10:g.',
    17: 'NC_000017.11:g.',
    18: 'NC_000018.10:g.',
    19: 'NC_000019.10:g.',
    20: 'NC_000020.11:g.',
    21: 'NC_000021.9:g.',
    22: 'NC_000022.11:g.',
    'X': 'NC_000023.11:g.',
    'Y': 'NC_000024.10:g.',
    'mt': 'NC_012920.1:g.',
    'NT_113889.1': 'NT_113889.1:g.',
    'NT_187633.1': 'NT_187633.1:g.',
    'NT_187661.1': 'NT_187661.1:g.',
    'NT_187693.1': 'NT_187693.1:g.',
    'NW_009646201.1': 'NW_009646201.1:g.'
}

In [6]:
variant_type_mapping = {'Duplication':"SO_1000035",
'copy_number_gain': "SO_0001742",
'copy_number_variation': "SO_0001019",
'herv_insertion': "SO_0002187",
'alu_insertion': "SO_0002063",
'Insertion': "SO_0000667",
'sva_insertion': "SO_0002065",
'line1_insertion': "SO_0002064",
'Deletion': "SO_0000159",
'copy_number_loss': "SO_0001743",
'alu_deletion': "SO_0002070",
'herv_deletion': "SO_0002067",
'line1_deletion': "SO_0002069",
'sva_deletion': "SO_0002068",
'Microsatellite': "SO_0000289",
'Inversion': "SO_1000036",
'single_nucleotide_variant':"SO_0001483",
'Indel': "SO_0002173",
'Variation': "SO_0000240"
}

In [7]:
clinical_signi_mapping = {
    'Benign': "NCIT_C14172",
    'Likely_benign': "NCIT_C168801",
    'Uncertain_significance': "NCIT_C94187",
    'Likely_pathogenic': "NCIT_C168800",
    'Pathogenic': "NCIT_C168799",
    'Likely_pathogenic,_low_penetrance': "NCIT_C202590",
    'Pathogenic, low penetrance': "",
    'Uncertain_risk_allele': "NCIT_C19785",
    'Likely_risk_allele': "",
    'Established_risk_allele': "",
    'drug_response': "NCIT_C142385",
    'association': "NCIT_C19402",
    'protective': "NCIT_C94434",
    'Affects': "NCIT_C64917",
    'other': "NCIT_C79397",
    'not_provided': "NCIT_C62635",
    'conflicting data from submitters': ""
    }

In [8]:
# Supongamos que tu dataframe se llama df y tu diccionario se llama descrip_alt_label_mapping
loaded_df['desc'] = loaded_df['CHROM'].map(descrip_alt_label_mapping)

In [9]:
loaded_df.index = loaded_df.index + 5521263

In [10]:
loaded_df['ID_base'] = 'VARHS' + (loaded_df.index + 1).astype(str)

# Paso 2: Identificar y manejar duplicados
# Crear un contador temporal para cada ID_base
loaded_df['temp_count'] = loaded_df.groupby('ID_base').cumcount() + 1

# Solo añadir el sufijo si hay más de una ocurrencia
loaded_df['ID_merged'] = loaded_df.apply(lambda x: f"{x['ID_base']}_{x['temp_count']}" if x['temp_count'] > 1 else x['ID_base'], axis=1)

# Eliminar la columna temporal usada para contar
loaded_df.drop(['temp_count'], axis=1, inplace=True)

columnas = ['ID_merged'] + [col for col in loaded_df.columns if col != 'ID_merged']
loaded_df = loaded_df[columnas]

In [11]:
loaded_df['CLNSIG_ID'] = loaded_df['CLNSIG'].map(clinical_signi_mapping)

In [12]:
# Añadir la columna 'DB' con el valor 'ClinVar' para todas las filas
loaded_df['DB'] = 'ClinVar'

In [13]:
loaded_df['variant_type_uri'] = loaded_df['CLNVC'].map(variant_type_mapping)

In [14]:
new_uri_mapping = {} # El objetivo es acaparar tanto unas URIs de los CHROM,  es que parece que pueden ser str o valores enteros

for k, v in chro_uri_mapping.items():
    if isinstance(k, int):  # Si la clave es entera, añadir tanto la versión entera como la versión string
        new_uri_mapping[k] = v
        new_uri_mapping[str(k)] = v
    else:  # Si la clave ya es un string, solo añadir esa clave
        new_uri_mapping[k] = v

In [15]:
# Añadir la columna 'CHROM_uri' con los valores mapeados desde la columna 'CHROM'
loaded_df['CHROM_uri'] = loaded_df['CHROM'].map(new_uri_mapping)

In [16]:
specific_row = loaded_df.iloc[2345]
print(specific_row)

ID_merged                                                VARHS5523610
CHROM                                                               1
POS                                                           1046641
ID                                                             967008
REF                                                                 G
ALT                                                                 A
ALLELEID                                                       951992
CLNDISDB            MONDO:MONDO:0014052,MedGen:C3808739,OMIM:61512...
CLNDN                                Congenital_myasthenic_syndrome_8
CLNHGVS                                     NC_000001.11:g.1046641G>A
CLNREVSTAT                        criteria_provided,_single_submitter
CLNSIG                                                  Likely_benign
CLNVC                                       single_nucleotide_variant
CLNVCSO                                                    SO:0001483
GENEINFO            

In [17]:
# Como tenemos valores NaN voy a ver el contenido de las lineas a ver que se debe y vemos si tienen algun valor distinto a NaN
if 'CLNDISDB' in loaded_df.columns:
    # Obtener los valores únicos en la columna 'SCIDN'
    unique_values = loaded_df['CLNDISDB'].dropna().unique()
    print("Valores únicos en la columna 'CLNDISDB':")
    print(unique_values)
else:
    print("La columna 'CLNDISDB' no existe en el DataFrame.") 

Valores únicos en la columna 'CLNDISDB':
['MeSH:D030342,MedGen:C0950123' 'MedGen:C3661900'
 'MeSH:D030342,MedGen:C0950123|MedGen:C3661900' ...
 'MONDO:MONDO:0010789,MedGen:C0162671,OMIM:540000,Orphanet:550|MedGen:C4016625'
 'MONDO:MONDO:0044970,MeSH:D028361,MedGen:C0751651,Orphanet:68380|Human_Phenotype_Ontology:HP:0003198,Human_Phenotype_Ontology:HP:0003569,Human_Phenotype_Ontology:HP:0003705,Human_Phenotype_Ontology:HP:0003742,Human_Phenotype_Ontology:HP:0003802,MONDO:MONDO:0005336,MeSH:D009135,MedGen:C0026848'
 'Human_Phenotype_Ontology:HP:0032224,MedGen:C0000778,OMIM:616093|MedGen:C3661900']


In [18]:
def extract_omim_ids(text):
    if pd.isna(text):
        return []  # Si el texto es NaN, retorna una lista vacía
    entries = str(text).split('|')
    omim_ids = []
    for entry in entries:
        items = entry.split(',')
        omim_items = [item.split(':')[1] for item in items if 'OMIM' in item]
        omim_ids.extend(omim_items)
    return omim_ids

# Aplicar la función y explotar los resultados en múltiples filas si es necesario
loaded_df['OMIM_ID'] = loaded_df['CLNDISDB'].apply(extract_omim_ids)
df_exploded = loaded_df.explode('OMIM_ID')

In [19]:
unique_omim_ids = df_exploded['MC_ID'].unique()
print(unique_omim_ids)

['SO:0001583' 'SO:0001819' 'SO:0001575' 'SO:0001627' 'SO:0001822'
 'SO:0001587' 'SO:0001589' 'SO:0001821' 'SO:0001574' 'SO:0002073' nan
 'SO:0001623' 'SO:0001624' 'SO:0001582' 'SO:0001820' 'SO:0001619'
 'SO:0001578' 'SO:0002152' 'SO:0002153']


In [20]:
# Como tenemos valores NaN voy a ver el contenido de las lineas a ver que se debe y vemos si tienen algun valor distinto a NaN
if 'OMIM_ID' in df_exploded.columns:
    # Obtener los valores únicos en la columna 'SCIDN'
    unique_values = df_exploded['OMIM_ID'].dropna().unique()
    print("Valores únicos en la columna 'OMIM_ID':")
    print(len(unique_values))
else:
    print("La columna 'OMIM_ID' no existe en el DataFrame.") 
    print(len(unique_values))

Valores únicos en la columna 'OMIM_ID':
7231


In [21]:
filtered_df = df_exploded[df_exploded['CLNDN'].str.contains('Aortic_aneurysm', na=False)]

# Verificar si el filtro ha encontrado alguna fila
if not filtered_df.empty:
    # Si quieres ver todas las filas que coinciden, puedes hacer:
    for index, row in filtered_df.iterrows():
        print(f"Fila {index}: {row['CLNDN']}")  # Puedes ajustar lo que necesites imprimir
else:
    print("No se encontraron filas que contengan 'Aortic_aneurysm' en la columna CLNDN.")

Fila 5600769: Congenital_primary_aphakia|Anterior_segment_dysgenesis|Aortic_aneurysm,_familial_thoracic_11,_susceptibility_to|Cardiovascular_phenotype
Fila 5600769: Congenital_primary_aphakia|Anterior_segment_dysgenesis|Aortic_aneurysm,_familial_thoracic_11,_susceptibility_to|Cardiovascular_phenotype
Fila 5600769: Congenital_primary_aphakia|Anterior_segment_dysgenesis|Aortic_aneurysm,_familial_thoracic_11,_susceptibility_to|Cardiovascular_phenotype
Fila 5600773: Aortic_aneurysm,_familial_thoracic_11,_susceptibility_to|Congenital_aneurysm_of_ascending_aorta|Acute_aortic_dissection
Fila 5600773: Aortic_aneurysm,_familial_thoracic_11,_susceptibility_to|Congenital_aneurysm_of_ascending_aorta|Acute_aortic_dissection
Fila 5600789: Anterior_segment_dysgenesis|Congenital_primary_aphakia|Aortic_aneurysm,_familial_thoracic_11,_susceptibility_to|Cardiovascular_phenotype|Acute_aortic_dissection|Congenital_aneurysm_of_ascending_aorta
Fila 5600789: Anterior_segment_dysgenesis|Congenital_primary_apha

In [22]:
regex = r"Aortic_aneurysm[^|]*"

# Filtrar el DataFrame para encontrar filas donde 'CLNDN' contiene formas variadas de 'Aortic_aneurysm'
filtered_df_2 = df_exploded[df_exploded['CLNDN'].str.contains(regex, na=False, case=False, regex=True)]

# Verificar si el filtro ha encontrado alguna fila y mostrar resultados
if not filtered_df_2.empty:
    print("Filas encontradas: ")
    print(filtered_df_2['CLNDN'])
else:
    print("No se encontraron filas que contengan variaciones de 'Aortic_aneurysm' en la columna CLNDN.")

Filas encontradas: 
5528057    Familial_thoracic_aortic_aneurysm_and_aortic_d...
5528057    Familial_thoracic_aortic_aneurysm_and_aortic_d...
5528058    Familial_thoracic_aortic_aneurysm_and_aortic_d...
5528058    Familial_thoracic_aortic_aneurysm_and_aortic_d...
5528063    Familial_thoracic_aortic_aneurysm_and_aortic_d...
                                 ...                        
8237280    Heterotopia,_periventricular,_X-linked_dominan...
8237280    Heterotopia,_periventricular,_X-linked_dominan...
8237280    Heterotopia,_periventricular,_X-linked_dominan...
8237280    Heterotopia,_periventricular,_X-linked_dominan...
8237280    Heterotopia,_periventricular,_X-linked_dominan...
Name: CLNDN, Length: 38942, dtype: object


In [23]:
specific_row = df_exploded.iloc[2316999]
print(specific_row)
omim_id_value = specific_row['CLNDISDB']
print(omim_id_value)

ID_merged                                                VARHS7504172
CHROM                                                              16
POS                                                           2064406
ID                                                              49162
REF                                                                 C
ALT                                                                 T
ALLELEID                                                        58324
CLNDISDB            MONDO:MONDO:0015356,MeSH:D009386,MedGen:C00276...
CLNDN               Hereditary_cancer-predisposing_syndrome|Tubero...
CLNHGVS                                     NC_000016.10:g.2064406C>T
CLNREVSTAT          criteria_provided,_multiple_submitters,_no_con...
CLNSIG                                                         Benign
CLNVC                                       single_nucleotide_variant
CLNVCSO                                                    SO:0001483
GENEINFO            

In [26]:
num_rows = df_exploded.shape[0]
print("Número de filas en el DataFrame:", num_rows)

Número de filas en el DataFrame: 3146279


In [27]:
num_rows = len(df_exploded)
print("Número de filas en el DataFrame:", num_rows)

Número de filas en el DataFrame: 3146279


In [28]:
# Define la ruta y el nombre del archivo TSV de salida
archivo_salida_tsv = 'salida_manejo_ClinVar.tsv'
carpeta_salida = 'Salidas'
archivo_salida_tsv = os.path.join(carpeta_salida, archivo_salida_tsv)
# Guardar el DataFrame en un archivo TSV
df_exploded.to_csv(archivo_salida_tsv, sep='\t', index=False)

print(f"DataFrame guardado con éxito en '{archivo_salida_tsv}'")

DataFrame guardado con éxito en 'Salidas\salida_manejo_ClinVar.tsv'


In [29]:
solo_tres_filas = df_exploded.head(1500)

# Guardar las primeras tres filas en un archivo CSV
solo_tres_filas.to_csv('primeras_tripletas_clinvar.tsv', index=False)  # index=False para no incluir el índice del DataFrame en el archivo

In [30]:
'''Consultas'''

'Consultas'

In [31]:
# Agrupa por 'variant_type' y selecciona los primeros 5 registros de cada grupo
consulta_1 = df_exploded.groupby('CLNVC').head(3)
print(consulta_1)

            ID_merged CHROM       POS       ID  \
5521264  VARHS5521265     1     69134  2205837   
5521265  VARHS5521266     1     69581  2252161   
5521266  VARHS5521267     1     69682  2396347   
5521286  VARHS5521287     1    930081  3038970   
5521288  VARHS5521289     1    930136  1658573   
5521306  VARHS5521307     1    930210  2059344   
5521311  VARHS5521312     1    930222   998906   
5521348  VARHS5521349     1    930328  2025454   
5521417  VARHS5521418     1    935845  1370201   
5521431  VARHS5521432     1    935866  1057892   
5521445  VARHS5521446     1    935909  1541010   
5521487  VARHS5521488     1    939146  1530016   
5521524  VARHS5521525     1    939398  1164669   
5521529  VARHS5521530     1    939398  2417769   
5521530  VARHS5521531     1    939398  1479161   
5521531  VARHS5521532     1    939398   769497   
5521542  VARHS5521543     1    939436  1404104   
5521692  VARHS5521693     1    942450  1634501   
5521693  VARHS5521694     1    942451   836156   


In [32]:
consulta_1.to_csv('consulta_1_clinVar.tsv', index=False)

In [33]:
consulta_2 = df_exploded[df_exploded['CLNVC'] == 'Deletion']

In [34]:
# Guarda el DataFrame filtrado en un archivo CSV
consulta_2.to_csv('consulta_2_clinVar.tsv', index=False)

In [35]:
# tamaño_porcion = 25000  # por ejemplo, 100 filas por archivo

# # Dividir el DataFrame en porciones más pequeñas
# lista_porcion_1 = [df_exploded.iloc[i:i + tamaño_porcion] for i in range(0, len(df_exploded), tamaño_porcion)]

# # Guardar cada porción en un archivo diferente
# for indice, porcion in enumerate(lista_porcion_1):
#     porcion.to_csv(f'consulta_2_parte_{indice + 1}.tsv', index=False, sep='\t')

In [36]:
# Filtra el DataFrame para obtener solo los registros donde gene es 'BRCA1'
consulta_3 = df_exploded[df_exploded['GENE_SYMBOLS'] == 'ACTA2']
print(consulta_3)

            ID_merged CHROM       POS       ID   REF ALT ALLELEID  \
6928702  VARHS6928703    10  88941034   674575  TCTC   T   664719   
6928703  VARHS6928704    10  88941211  2911389     A   G  3078741   
6928704  VARHS6928705    10  88941219  2173773     C   T  1897443   
6928705  VARHS6928706    10  88941223   838562     C   T   852281   
6928705  VARHS6928706    10  88941223   838562     C   T   852281   
...               ...   ...       ...      ...   ...  ..      ...   
6929010  VARHS6929011    10  88952822   301513     T   C   311581   
6929010  VARHS6929011    10  88952822   301513     T   C   311581   
6929010  VARHS6929011    10  88952822   301513     T   C   311581   
6929011  VARHS6929012    10  88952974  1243600     A   G  1233088   
6929012  VARHS6929013    10  88953010  1321521     C   G  1311803   

                                                  CLNDISDB  \
6928702                                    MedGen:CN517202   
6928703    MONDO:MONDO:0012730,MedGen:C2673186,

In [37]:
# Guarda el DataFrame filtrado en un archivo CSV llamado 'consulta_3.csv'
consulta_3.to_csv('consulta_3_clinVar.tsv', index=False)

In [38]:
filtered_df = df_exploded[df_exploded['CLNDN'] == 'Aortic_aneurysm']

# Verificar si el filtro ha encontrado alguna fila
if not filtered_df.empty:
    # Si solo quieres la primera fila que coincide
    specific_row = filtered_df.iloc[1]
    print(specific_row)
else:
    print("No se encontraron filas con 'Aortic_aneurysm' en la columna CLNDN.")

ID_merged                                                VARHS7445876
CHROM                                                              15
POS                                                          67066155
ID                                                            1172631
REF                                                                 A
ALT                                                                 G
ALLELEID                                                      1161836
CLNDISDB            Human_Phenotype_Ontology:HP:0004942,MONDO:MOND...
CLNDN                                                 Aortic_aneurysm
CLNHGVS                                    NC_000015.10:g.67066155A>G
CLNREVSTAT                        criteria_provided,_single_submitter
CLNSIG                                                     Pathogenic
CLNVC                                       single_nucleotide_variant
CLNVCSO                                                    SO:0001483
GENEINFO            

In [39]:
#Buscar la enfermedad X y determinar el tipo de variante más comúnmente asociada a ella y el cromosoma donde esta variante es más frecuente." (no funciona) donde X es el nombre de la enfermedad
# Asegúrate de que la columna CLNDN es de tipo string; si no es así, puedes convertirla.
df_exploded['CLNDN'] = df_exploded['CLNDN'].astype(str)

# Filtra el DataFrame para obtener solo los registros donde CLNDN contiene 'Aortic_aneurysm'
consulta_4 = df_exploded[df_exploded['CLNDN'].str.contains('Aortic_aneurysm')]

In [40]:
# Guarda el DataFrame filtrado en un archivo CSV llamado 'consulta_4.csv'
consulta_4.to_csv('consulta_4_clinVar.tsv', index=False)
print(consulta_4)

            ID_merged CHROM        POS      ID REF ALT ALLELEID  \
5600769  VARHS5600770     1   47416719  650898   A   G   628136   
5600769  VARHS5600770     1   47416719  650898   A   G   628136   
5600769  VARHS5600770     1   47416719  650898   A   G   628136   
5600773  VARHS5600774     1   47416725  375652   G   A   362493   
5600773  VARHS5600774     1   47416725  375652   G   A   362493   
...               ...   ...        ...     ...  ..  ..      ...   
8235580  VARHS8235581     X  154359631  264534   T   C   259161   
8235580  VARHS8235581     X  154359631  264534   T   C   259161   
8235580  VARHS8235581     X  154359631  264534   T   C   259161   
8235580  VARHS8235581     X  154359631  264534   T   C   259161   
8235580  VARHS8235581     X  154359631  264534   T   C   259161   

                                                  CLNDISDB  \
5600769  MONDO:MONDO:0012456,MedGen:C1853230,OMIM:61025...   
5600769  MONDO:MONDO:0012456,MedGen:C1853230,OMIM:61025...   
5600769  

In [41]:
tamaño_porcion = 2500  # por ejemplo, 100 filas por archivo

# Dividir el DataFrame en porciones más pequeñas
lista_porcion = [df_exploded.iloc[i:i + tamaño_porcion] for i in range(0, len(df_exploded), tamaño_porcion)]

# Guardar cada porción en un archivo diferente
for indice, porcion in enumerate(lista_porcion):
    porcion.to_csv(f'consulta_4_parte_{indice + 1}.tsv', index=False, sep='\t')

In [42]:
# Filtra el DataFrame para obtener solo los registros donde CHROM es igual a 17
consulta_5 = df_exploded[df_exploded['CHROM'] == '17']
print(consulta_5)

            ID_merged CHROM       POS       ID REF ALT ALLELEID  \
7629837  VARHS7629838    17    138238  2391745   A   C  2381623   
7629838  VARHS7629839    17    160690  2647151   C   G  2814629   
7629839  VARHS7629840    17    161456  2245062   G   A  2240966   
7629840  VARHS7629841    17    161480  2606353   G   A  2768784   
7629841  VARHS7629842    17    161525  2392654   C   T  2375052   
...               ...   ...       ...      ...  ..  ..      ...   
7799980  VARHS7799981    17  83094397  2346242   G   A  2341264   
7799981  VARHS7799982    17  83094427  2243412   G   A  2230246   
7799982  VARHS7799983    17  83094430  2382272   C   T  2372171   
7799983  VARHS7799984    17  83094560  2548448   G   C  2706869   
7799984  VARHS7799985    17  83094568  2516321   C   T  2674703   

                             CLNDISDB                    CLNDN  \
7629837  MeSH:D030342,MedGen:C0950123  Inborn_genetic_diseases   
7629838               MedGen:C3661900             not_provided 

In [43]:
# Guarda el DataFrame filtrado en un archivo CSV llamado 'consulta_5.csv'
consulta_5.to_csv('consulta_5_clinVar.tsv', index=False)